In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15968571
paper_name = 'delia_hammond_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [23]:
files = ['hits.txt','hits2.txt']
seps = [' ','\t']

In [24]:
original_data_list = []
for ixf, f in enumerate(files):
    original_data = pd.read_csv('raw_data/' + f, header=None, names=['orf','data'], sep=seps[ixf])
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['orf'] = original_data['orf'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data['data'] = pd.to_numeric(original_data['data'], errors='coerce')
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    original_data_list.append(original_data)

Original data dimensions: 40 x 2
       orf   data
0  YOL102C  0.903
1  YOR310C  0.900
2  YKR081C  0.897
3  YHR142W  0.884
4  YHR151C  0.870
Empty DataFrame
Columns: [orf, data]
Index: []
(40, 1)
Original data dimensions: 20 x 2
       orf   data
0  YHL030W  72.32
1  YBR257W  67.33
2  YGR019W  23.73
3  YBR256C  23.35
4  YBL101C  12.92
Empty DataFrame
Columns: [orf, data]
Index: []
(20, 1)


In [31]:
original_data = pd.concat(original_data_list, axis=1)
original_data.index.name='orf'

In [32]:
original_data[original_data.isnull()] = 0

In [33]:
original_data = original_data.groupby(original_data.index).mean()
original_data.shape

(60, 2)

# Prepare the final dataset

In [35]:
data = original_data.copy()

In [36]:
dataset_ids = [495, 5006]
datasets = datasets.reindex(index=dataset_ids)

In [37]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [38]:
data.head()

dataset_id,495,5006
data_type,value,value
orf,,
YBL092W,-0.891,0.00
YBL101C,0.000,12.92
YBR256C,0.000,23.35
YBR257W,0.000,67.33
YCL009C,-0.885,0.00


## Subset to the genes currently in SGD

In [39]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [40]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,495,5006
,data_type,value,value
gene_id,orf,,
180,YBL092W,-0.891,0.00
189,YBL101C,0.000,12.92
450,YBR256C,0.000,23.35
451,YBR257W,0.000,67.33
503,YCL009C,-0.885,0.00


# Normalize

In [41]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [42]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [43]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,495,5006,495,5006
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
180,YBL092W,-0.891,0.00,-11.238903,0.000000
189,YBL101C,0.000,12.92,-0.051366,8.175427
450,YBR256C,0.000,23.35,-0.051366,14.775249
451,YBR257W,0.000,67.33,-0.051366,42.604603
503,YCL009C,-0.885,0.00,-11.163566,0.000000


# Print out

In [44]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [45]:
from IO.save_data_to_db3 import *

In [46]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 15968571...
Inserting the new data...


100%|██████████| 2/2 [00:00<00:00,  5.84it/s]


Updating the data_modified_on field...
